In [3]:
import os
root_dir = f"{os.getcwd().split('TSM-Bench')[0]}/TSM-Bench"
os.chdir(root_dir)
os.getcwd()

'/home/luca/test/TSM-Bench'

In [4]:
import os
import pandas
import systems

from systems import (clickhouse, questdb , extremedb, druid, timescaledb, influx , monetdb)


In [5]:
def load_query_file(system):
    path = os.path.dirname(system.__file__)
    with open(f'{path}/queries.sql') as file: 
        queries = [line.rstrip() for line in file]
    return queries

In [20]:
# settings
n_sensors = [1]#,10,20,30,40,50,60,70,80,90,100]
n_stations =  [1,2]#,3,4,5,6,7,8,9,10] 
time_ranges = ["minute","hour","day","week","month"]
dataset = "d1"

n_iter = 3#2000
timeout = 5000

scenarios  = [(sensor, station, time_range) for sensor in n_sensors for station in n_stations for time_range in time_ranges]
scenarios

[(1, 1, 'minute'),
 (1, 1, 'hour'),
 (1, 1, 'day'),
 (1, 1, 'week'),
 (1, 1, 'month'),
 (1, 2, 'minute'),
 (1, 2, 'hour'),
 (1, 2, 'day'),
 (1, 2, 'week'),
 (1, 2, 'month')]

In [21]:
def load_query_file(system):
    path = os.path.dirname(system.__file__)
    with open(f'{path}/queries.sql') as file: 
                queries = [line.rstrip() for line in file]
    return queries


map = {
    0 : float ,
    1 : float,
    2 : int , #query
    3:  int , #sensors
    4: int ,  #stations
    5 : str,  #time
    6 : str   #system
}

output_file = "results/parameters.txt"

def get_already_computed(file):
    computed_set = set()
    with open(file, "r") as f:
        for line in f:
            try:
                t = tuple( (map[i](f.strip()) for i,f in enumerate( line.strip().split(','))))
                computed_set.add(t)
            except:
                #print(line)
                pass
    return computed_set




In [29]:
systems = (clickhouse,questdb, clickhouse , extremedb, timescaledb, monetdb)
output_file = "results/parameters.txt"

computed_set = get_already_computed(output_file)
computed_set = set( tuple(values) for _,_,*values in  computed_set)

for system in systems:
    system.launch()
    system_name = system.__file__.split("/")[-2]
    queries = load_query_file(system)
    print(queries)
    for i ,query in enumerate(queries):
        query = query.replace("<db>", dataset)
        if "select" not in query.lower():
            continue
        print("query" , query)
        for n_s, n_st, time_range in scenarios:
            if (i,n_s,n_st,time_range,system) in computed_set:
                print("already computed")
                continue
            try:
                time , var = system.run_query(query , rangeUnit=time_range , rangeL = 1 , n_s=n_s , n_it=n_iter , n_st = n_st) 
            except Exception as E:
                print(E)
                print("query" , query)
                time , var = "fail" , str(E)
                raise E
            with open(output_file, "a") as file:
                line = f"{time} , {var}  , {i} , {n_s} , {n_st} , {time_range} , {system_name} \n"
                file.write(line)
    system.stop()

/home/luca/test/TSM-Bench/systems/clickhouse/run_system.py
changing directory to /home/luca/test/TSM-Bench/systems/clickhouse
["select time, id_station, <sid> FROM <db> where id_station in <stid> AND time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>';", "select time, id_station, <sid> FROM <db> where id_station in <stid> AND time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' and <sfilter>;", "SELECT id_station, <avg_s> FROM <db> WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' AND id_station in <stid> GROUP BY id_station;", 'SELECT id_station, EXTRACT(YEAR FROM time) AS "year", date_trunc(\'month\', time) AS "month", date_trunc(\'day\', time) AS "day", date_trunc(\'hour\', time) AS "hour", <avg_s> FROM <db> where  time > TIMESTAMP \'<timestamp>\' - INTERVAL \'<range>\' <rangesUnit> AND time < TIMESTAMP \'<timestamp>\' AND id_station 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.58s/it]


query select time, id_station, <sid> FROM d1 where id_station in <stid> AND time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' and <sfilter>;


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.91it/s]


query SELECT id_station, <avg_s> FROM d1 WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' AND id_station in <stid> GROUP BY id_station;


...

query with data1 as ( select time, s<sid1> as s_1 from d1 WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' AND id_station='st<stid1>' ), data2 as ( select time, s<sid2> as s_2 from d1 WHERE time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>' AND id_station='st<stid2>' ) select data1.time, s_1, s_2, (s_1 + s_2)/2 as avg from data1,data2 where data1.time = data2.time;


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:05<00:00,  1.86s/it]


query SELECT sys.corr(s<sid1>, s<sid2>) AS pearson_corr FROM d1 WHERE id_station=<stid> AND time > TIMESTAMP '<timestamp>' - INTERVAL '<range>' <rangesUnit> AND time < TIMESTAMP '<timestamp>';


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 15.04it/s]

/home/luca/test/TSM-Bench/systems/monetdb/run_system.py
changing directory to /home/luca/test/TSM-Bench/systems/monetdb


In [ ]:
for (n_s, n_st, time_range) in list(scenarios):
    print(n_s)

In [ ]:
list(scenarios)